#### 同策略基金分析

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from function import *
from WindPy import w
w.start()

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
# 策略名称
strategy = "量化CTA"
# 辅助日期
year_begin_date = "2023-12-29"
month_begin_date = "2024-11-29"
week_begin_date = "2024-11-29"

In [3]:
basic_info = load_data("产品目录.xlsx")
strategy_info = basic_info.loc[(basic_info["strategy"] == strategy), :]
strategy_info.reset_index(drop=True, inplace=True)

In [10]:
strategy_info

,strategy,fund_code,fund_name,benchmark_code,benchmark_name,threshold
0,量化CTA,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.1
1,量化CTA,SXF557,千衍九凌1号,NH0100.NHF,南华商品指数,-0.1
2,量化CTA,SCV067,众壹资产量合兴成一号,NH0100.NHF,南华商品指数,-0.1
3,量化CTA,SLS182,弘源多元化CTA,NH0100.NHF,南华商品指数,-0.1
4,量化CTA,SN3221,宏锡量化CTA7号,NH0100.NHF,南华商品指数,-0.1
5,量化CTA,SGA623,会世元丰CTA1号,NH0100.NHF,南华商品指数,-0.1
6,量化CTA,SAET98,量派CTA七号,NH0100.NHF,南华商品指数,-0.1
7,量化CTA,S83994,远澜云杉,NH0100.NHF,南华商品指数,-0.1


In [6]:
nav_df = load_data("data\均成CTA1号净值20241224.xlsx")

In [8]:
nav_df = nav_df.sort_values(by="日期", ascending=True)

In [9]:
nav_df

,日期,单位净值,累计净值,复权净值,涨跌幅
1504,2017-03-01,1.0000,1.0000,1.0000,--
1503,2017-03-02,1.0001,1.0001,1.0001,0.01%
1502,2017-03-03,1.0001,1.0001,1.0001,0.00%
1501,2017-03-06,1.0002,1.0002,1.0002,0.01%
1500,2017-03-07,1.0002,1.0002,1.0002,0.00%
...,...,...,...,...,...
4,2024-11-22,6.7960,7.4410,7.8154,-0.26%
3,2024-11-29,6.6780,7.3230,7.6797,-1.74%
2,2024-12-06,6.7290,7.3740,7.7383,0.76%
1,2024-12-13,6.6310,7.2760,7.6257,-1.46%


#### 表格

In [4]:
nav_dfs = None
nav_table = pd.DataFrame(
    columns=[
        "产品名称",
        "日期",
        "单位净值",
        "累计净值",
        "复权净值",
        "本年收益(%)",
        "当月收益(%)",
        "当周收益(%)",
    ]
)
for fund in strategy_info["fund_name"]:
    # 导入净值数据
    nav_df_path = os.path.join(
        r"data", f"{fund}_{"净值20241224"}.xlsx"
    )
    nav_df = load_data(nav_df_path)
    nav_df = nav_df.sort_values(by="日期", ascending=True)
    nav_df = get_standardized_data(nav_df)
    nav_df.rename(
        columns={
            "date": "日期",
            "nav_unit": "单位净值",
            "nav_accumulated": "累计净值",
            "nav_adjusted": "复权净值",
        },
        inplace=True,
    )
    # 计算年、月、周收益
    year_begin_nav = nav_df[nav_df["日期"] == year_begin_date]
    month_begin_nav = nav_df[nav_df["日期"] == month_begin_date]
    week_begin_nav = nav_df[nav_df["日期"] == week_begin_date]
    nav_head = nav_df.head(1)
    nav_tail = nav_df.tail(1)
    nav_tail["产品名称"] = fund
    if not year_begin_nav.empty:
        year_begin_value = year_begin_nav["复权净值"].iloc[0]
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / year_begin_value) - 1) * 100, 2
        )
    else:  # 如果找不到2023-12-29净值，则使用第一条数据
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / nav_head["复权净值"].iloc[0]) - 1) * 100, 2
        )
    month_begin_value = month_begin_nav["复权净值"].iloc[0]
    nav_tail["当月收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / month_begin_value) - 1) * 100, 2
    )
    week_begin_value = week_begin_nav["复权净值"].iloc[0]
    nav_tail["当周收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / week_begin_value) - 1) * 100, 2
    )
    # 添加至表格
    nav_table = pd.concat([nav_table, nav_tail], ignore_index=True)

In [5]:
nav_table

,产品名称,日期,单位净值,累计净值,复权净值,本年收益(%),当月收益(%),当周收益(%)
0,均成CTA1号,2024-12-06,6.7290,7.3740,7.738350,8.11,0.76,0.76
1,弘源多元化CTA,2024-12-06,1.6530,1.6530,1.653000,15.27,1.35,1.35
2,宏锡量化CTA7号,2024-12-06,1.2718,2.1940,2.799434,12.28,1.34,1.34
3,会世泰和CTA1号,2024-12-06,2.1489,2.1489,2.148900,19.62,2.27,2.27
4,量派CTA一号,2024-12-06,1.2239,1.2239,1.223900,22.39,2.07,2.07
5,远澜云杉,2024-12-06,2.8760,2.8760,2.876000,2.09,0.74,0.74


#### 折线图

In [6]:
nav_dfs = None
for fund in strategy_info["fund_name"]:
    nav_data_path = os.path.join(r"data",f"{strategy_info.loc[0,"strategy"]}_{fund}.xlsx")
    nav_df = load_data(nav_data_path)
    nav_df = get_standardized_data(nav_df)
    nav_df.rename(columns={"date":"日期", "nav_adjusted":fund}, inplace=True)
    nav_df = nav_df[["日期",fund]]
    if nav_dfs is None:
        nav_dfs = nav_df
    else:
        nav_dfs = pd.merge(nav_dfs, nav_df, on="日期", how="inner")
        nav_dfs = nav_dfs.set_index("日期")
        nav_dfs = nav_dfs.div(nav_dfs.iloc[0])

In [7]:
get_line(nav_dfs, {strategy_info.loc[0,"strategy"]}).render_notebook()